## Foursquare API  to .csv file

In [2]:
import foursquare
import json, requests
import urllib
import pandas as pd
import numpy as np
import pprint as pprint
import time
from tqdm import *



## Foursquare's API 
- Client ID and Secret .....

In [10]:
# %load ./../../../../key/foursquareapi.py

CLIENT_SECRET1 = 'AECR3IQNZBLE5TGED2UK3J0MFQWURO33JRM4RMMFVGYRQ13L'
CLIENT_ID1 = 'SSME5S2T2JOAUIT1XM1HGDHG5SH10EHS2TFAMUXJQNQGOUSS5'


CLIENT_SECRET2 = 'IYHEJAK0VHCDLI3O4FG2NWSQ54WJPKIBGAZU4PGKP3E3044S'
CLIENT_ID2 = 'F41SQEZKJSIOJY03ODBHYL4BKPEK5QM1UYD3BPGLHMHJDUTV'


## Test Response
- gathered data from foursquare api and save to new csv 

- inital test to check response  

In [13]:
# First Request Test
url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
  client_id= CLIENT_ID2,
  client_secret= CLIENT_SECRET2,
  v='20170801',
  ll='51.5074,0.1278',
  section = 'topPick',
  limit=100
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
# raw = response.read().decode('utf8')?????
resp

<Response [200]>

In [5]:
# http://www.nltk.org/book/ch03.html decodeeeeeee

## Categories 
- I decided to choose different categories.  
- use of csv category from foursquare 
https://developer.foursquare.com/docs/resources/categories

In [116]:
cat = pd.read_csv('./raw_data/category_tree.csv')
cat.head(3)

,level1_name,level1_id,level2_name,level2_id,level3_name,level3_id,level4_name,level4_id
0,Arts & Entertainment,4d4b7104d754a06370d81259,Amphitheaters,56aa371be4b08b9a8d5734db,NaN,NaN,NaN,NaN
1,Arts & Entertainment,4d4b7104d754a06370d81259,Aquariums,4fceea171983d5d06c3e9823,NaN,NaN,NaN,NaN
2,Arts & Entertainment,4d4b7104d754a06370d81259,Arcades,4bf58dd8d48988d1e1931735,NaN,NaN,NaN,NaN


In [117]:
# keep only necessary columns
cat = cat[['level1_name', 'level2_id', 'level2_name']]
cat.columns = ['cat_parent', 'cat_id', 'cat_name']
cat = categories.dropna(axis=0)
cat.head()

,cat_parent,cat_id,cat_name
642,Shops & Services,52f2ab2ebcbc57f1066b8b56,ATMs
638,Shops & Services,5267e446e4b0ec79466e48c4,Adult Boutiques
111,Food,503288ae91d4c4b30a586d67,Afghan Restaurants
113,Food,4bf58dd8d48988d1c8941735,African Restaurants
112,Food,4bf58dd8d48988d1c8941735,African Restaurants


In [119]:
cat.cat_parent.unique()

array(['Shops & Services', 'Food', 'Travel & Transport',
       'Arts & Entertainment', 'Professional & Other Places',
       'Residences', 'Outdoors & Recreation', 'Nightlife Spots', 'Events',
       'Colleges & Universities'], dtype=object)

In [120]:
food_cat = cat[cat['cat_parent'] == 'Food']
print food_cat.shape
food_cat.head()

(304, 3)


,cat_parent,cat_id,cat_name
111,Food,503288ae91d4c4b30a586d67,Afghan Restaurants
113,Food,4bf58dd8d48988d1c8941735,African Restaurants
112,Food,4bf58dd8d48988d1c8941735,African Restaurants
114,Food,4bf58dd8d48988d14e941735,American Restaurants
115,Food,4bf58dd8d48988d14e941735,American Restaurants


In [130]:
sub_cat_id = food_cat.cat_id.unique()
len(sub_cat_id)

89

## Long Lat Tube Stations 
- csv of locations of tube stations, source from doogal which specialises in UK postcodes and geolocations and map data.
https://www.doogal.co.uk/london_stations.php


In [127]:
stations_raw = pd.read_csv('./raw_data/London stations.csv')
# only stations zone 1-4
stations = stations_raw[(stations_raw['Zone'] == "4") |(stations_raw['Zone'] == "3")|
                       (stations_raw['Zone'] == "2")|(stations_raw['Zone'] == "1")]

stations.head(3)

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton,520296,181196,51.516887,-0.267690,3,W3 0BP


In [128]:
latlong = list(stations.Latitude.apply(str) + ',' + stations.Longitude.apply(str))

##  Categories in Locations 
- parameter combinations

In [131]:
len(latlong)*len(sub_cat_id)

36935

In [133]:
all_params = []
for ll in latlong:
    for cat in sub_cat_id:   
        all_params.append(dict( client_id= CLIENT_ID2, client_secret= CLIENT_SECRET2,
                         ll= ll,
                         radius = 250,
                         v='20171001',
                         limit = 100, 
                         intent='browse',
                         categoryId = cat))    
print all_params[5]
print len(all_params)

{'ll': '51.5319519867,0.00372337109717', 'limit': 100, 'client_id': 'F41SQEZKJSIOJY03ODBHYL4BKPEK5QM1UYD3BPGLHMHJDUTV', 'v': '20171001', 'radius': 250, 'client_secret': 'IYHEJAK0VHCDLI3O4FG2NWSQ54WJPKIBGAZU4PGKP3E3044S', 'intent': 'browse', 'categoryId': '52e81612bcbc57f1066b7a01'}
36935


# Restaurant Search 
- 36935 parameters 
- reqested from API and saved straight to json files

In [141]:
last=all_params[-10000:]
url_london = 'https://api.foursquare.com/v2/venues/search'

data = []
i = 0

for params in tqdm(last):     
        time.sleep(.02)
        resp = requests.get(url=url_london, params=params)
        data.append(json.loads(resp.text))
        
        i+=1
        
        if i % 10000 == 0:
            title = 'last_saved_dump_{0}.json'.format(i)   
            d = json.dumps(data)
            with open ("./data_params/food/"+ title, "w") as f:
                f.write(d)          
            
            print title
            data = []

100%|██████████| 10000/10000 [36:15<00:00,  4.60it/s]

last_saved_dump_10000.json


In [139]:
url_london = 'https://api.foursquare.com/v2/venues/search'

data = []
i = 0

for params in tqdm(all_params):     
        time.sleep(.02)
        resp = requests.get(url=url_london, params=params)
        data.append(json.loads(resp.text))
        
        i+=1
        
        if i % 10000 == 0:
            title = 'saved_dump_{0}.json'.format(i)   
            d = json.dumps(data)
            with open ("./data_params/food/"+ title, "w") as f:
                f.write(d)          
            
            print title
            data = []

 27%|██▋       | 10001/36935 [29:37<1:19:47,  5.63it/s]

saved_dump_10000.json


 54%|█████▍    | 20000/36935 [57:27<48:39,  5.80it/s]  

saved_dump_20000.json


 81%|████████  | 30001/36935 [1:28:00<20:20,  5.68it/s]

saved_dump_30000.json


100%|██████████| 36935/36935 [1:49:39<00:00,  5.61it/s]


# Test Venue Search 
- 1050 parameters 
- reqested from API and saved straight to json files

In [24]:
test_params = all_params[1000:2050]

In [25]:
url_london = 'https://api.foursquare.com/v2/venues/search'

data = []
i = 0

for params in tqdm(test_params):     
        time.sleep(.02)
        resp = requests.get(url=url_london, params=params)
        data.append(json.loads(resp.text))
        
        i+=1
        
        if i % 100 == 0:
            title = 'saved_dump_{0}.json'.format(i)   
            d = json.dumps(data)
            with open ("./data_params/test/"+ title, "w") as f:
                f.write(d)          
                
            print title
            data = []

 10%|▉         | 100/1050 [00:20<03:10,  4.99it/s]

saved_dump_100.json


 19%|█▉        | 200/1050 [00:42<03:01,  4.69it/s]

saved_dump_200.json


 29%|██▊       | 301/1050 [01:03<02:38,  4.73it/s]

...

 86%|████████▌ | 901/1050 [03:04<00:30,  4.89it/s]

saved_dump_900.json


 95%|█████████▌| 1001/1050 [03:23<00:09,  4.92it/s]

saved_dump_1000.json


100%|██████████| 1050/1050 [03:33<00:00,  4.91it/s]


## Complete Venues Search
- Obtaining requests and json files for each parameter combination
- only 100,000 requests every 24 hours 
- so separation of parameters and running will be done separately

In [26]:
len(all_params)

175960

In [27]:
first = all_params[:90000]
second = all_params[90001 :]

In [28]:
# first 100000 requests
url_london = 'https://api.foursquare.com/v2/venues/search'

data = []
i = 0

for params in tqdm(first):     
        time.sleep(.02)
        resp = requests.get(url=url_london, params=params)
        data.append(json.loads(resp.text))
        
        i+=1
        
        if i % 10000 == 0:
            title = 'saved_dump_{0}.json'.format(i)   
            d = json.dumps(data)
            with open ("./data_params/"+ title, "w") as f:
                f.write(d)
                
            print title
            data = []

 11%|█         | 10000/90000 [33:16<4:26:10,  5.01it/s]

saved_dump_10000.json


 22%|██▏       | 20000/90000 [1:06:20<3:52:11,  5.02it/s]

saved_dump_20000.json


 33%|███▎      | 30001/90000 [1:41:33<3:23:06,  4.92it/s]

...

saved_dump_70000.json


 89%|████████▉ | 80001/90000 [4:25:42<33:12,  5.02it/s]  

saved_dump_80000.json


100%|██████████| 90000/90000 [4:58:24<00:00,  5.03it/s]

saved_dump_90000.json


In [ ]:
#             for the last 50
#             if i > 1000:
#                 title = 'saved_dump_1100.json'  
#                 d = json.dumps(resp.text)
#                 with open ("./data_params/"+ title, "w") as f:
#                     f.write(d)              

In [ ]:
# data = []
# data.append(json.loads(resp.text))

lonresp = requests.get(url=url_london, params=london_params, timeout=2)
londata = json.loads(resp.text)

### Make DataFrame and save as a CSV

In [ ]:
cols = ['venue_name', 'venue_id','rating','tips', 'cat_name','cat_id','ll']
venue_dict = {'venue_name': venue_name, 'venue_id':venue_id,'rating':rating, 'tips':tips, 'cat_name':cat_name,'cat_id':cat_id,'ll':ll, 'lng':lng, 'lat':lat}

venues = pd.DataFrame(venue_dict, columns = cols)
venues.to_csv('londonvenues.csv', header=True, index=False, encoding='utf-8')
venues.shape

In [ ]:
venue = pd.read_csv('./londonvenues.csv')
venue.head()

In [ ]:
venue.shape

In [ ]:
venue.isnull().sum()

### Document your work so far...
Once you have your data, read into it and review it to confirm whether it is as productive as you intended. If not, switch datasets, gather additional data (e.g. multiple datasets), or revise your project goals.
- for each category of 424 types. 

Create your own database? 

2. Perform preliminary data munging and cleaning of your data: organize your data relevant to your project goals.
   - Review data to verify initial assumptions
   - Clean and munge data as necessary
3. Describe your data: keep your intended audience(s) in mind.
   - Document your work so far in a Jupyter notebook.



   - Outline proposed methods and models
   - Define risks & assumptions

5. Create a blog post of at least 500 words that describes your work so far. Link to it in your Jupyter notebook.

## Useful Resources

- [Exploratory Data Analysis](http://insightdatascience.com/blog/eda-and-graphics-eli-bressert.html)
- [Best practices for data documentation](https://www.dataone.org/all-best-practices)
